# Notebook 01: Pydantic para Pipelines de AI/ML

**Clase extra de Python** | Notebook 6 de la serie

---

## Objetivos de aprendizaje

Al finalizar este notebook seras capaz de:

1. **Validar requests de inferencia** usando modelos Pydantic con restricciones de campo.
2. **Crear modelos con `field_validator`** para reglas de negocio personalizadas.
3. **Manejar errores de validacion** de forma estructurada y accionable.
4. **Comparar dict vs Pydantic** y entender por que los dicts producen bugs silenciosos.
5. **Disenar contratos de datos** para pipelines de AI/ML completos.

---

| Aspecto | Detalle |
|---|---|
| **Tiempo estimado** | 60-90 minutos |
| **Dependencias** | `pydantic` (ya instalado en el proyecto) |
| **API keys** | No requiere. Todo se ejecuta localmente. |
| **Base** | Expande `ejemplo_07_pydantic_ai.py` |

> **Nota:** Este notebook no requiere API keys ni conexion a internet. Todo el codigo se ejecuta localmente.

In [ ]:
# === SETUP ===

from __future__ import annotations

from typing import Literal, Optional
from importlib import import_module

from pydantic import (
    BaseModel,
    Field,
    ValidationError,
    field_validator,
    model_validator,
)

import json
from datetime import datetime

print("Setup completo")
print(f"Pydantic version: {import_module('pydantic').VERSION}")
print(f"Fecha de ejecucion: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

## Por que validar datos en AI/ML?

En un pipeline de AI/ML, los datos pasan por multiples etapas. **Sin validacion**, los errores
se propagan silenciosamente y explotan en los lugares mas dificiles de depurar.

### Flujo tipico de un pipeline

```
  Datos crudos       Validacion       Preprocesamiento       Modelo        Validacion       Output
  (JSON/API)         de entrada       (features)             (inferencia)  de salida        (respuesta)
       |                 |                  |                    |               |               |
       v                 v                  v                    v               v               v
  +----------+     +-----------+     +--------------+     +-----------+   +-----------+   +-----------+
  | raw_data | --> | Pydantic  | --> | transform()  | --> | predict() | ->| Pydantic  | ->|  cliente  |
  | (dict)   |     | Model     |     | normalize()  |     |           |   | Model     |   |  recibe   |
  +----------+     +-----------+     +--------------+     +-----------+   +-----------+   +-----------+
       ^                 ^                  ^                    ^               ^               ^
     Puede            CAPTURA            Puede               Puede           CAPTURA          Datos
     tener            errores            fallar si            dar             errores          limpios
     basura           AQUI               datos malos          basura          AQUI             y seguros
```

### Que puede salir mal SIN validacion?

| Etapa | Sin validacion | Con Pydantic |
|---|---|---|
| **Input** | Tipos incorrectos pasan silenciosamente (`"123"` como string en lugar de int) | `ValidationError` inmediato con ubicacion exacta del error |
| **Features** | Valores fuera de rango (`age = -5`) corrompen el modelo | Restricciones `ge=0, le=120` rechazan el dato |
| **Output del modelo** | Score de 1.5 (fuera de [0,1]) se envia al cliente | Constraint `le=1.0` lo detecta antes de responder |
| **Campos faltantes** | `KeyError` en produccion a las 3 AM | Campo requerido reportado con nombre y tipo esperado |

> **Principio clave:** Valida en los bordes del sistema (entrada y salida). El interior puede confiar en datos limpios.

---

## Seccion 1: Validacion de Request de Inferencia

Cuando un cliente envia un request a tu API de inferencia, necesitas garantizar que:

- El **prompt** tenga una longitud razonable (ni vacio ni excesivamente largo).
- Los **hiperparametros** (`max_tokens`, `temperature`) esten dentro de rangos validos.
- Los **tipos** sean correctos (no aceptar strings donde esperamos numeros).

Pydantic hace todo esto de forma declarativa con `Field()`.

In [ ]:
# --- Modelo: InferenceRequest ---

class InferenceRequest(BaseModel):
    """Valida un request de inferencia a un LLM."""
    prompt: str = Field(min_length=5, max_length=2000, description="Texto de entrada para el modelo")
    max_tokens: int = Field(default=256, ge=1, le=4096, description="Maximo de tokens a generar")
    temperature: float = Field(default=0.2, ge=0.0, le=2.0, description="Temperatura de muestreo")


# --- Demo: request VALIDO ---
print("=" * 60)
print("REQUEST VALIDO")
print("=" * 60)

valid_payload = {
    "prompt": "Explica en 3 puntos por que MLOps necesita validacion de datos.",
    "max_tokens": 300,
    "temperature": 0.3,
}

request = InferenceRequest(**valid_payload)
print(f"Tipo: {type(request).__name__}")
print(f"Datos validados: {request.model_dump()}")
print()

# --- Demo: request INVALIDO ---
print("=" * 60)
print("REQUEST INVALIDO (3 errores simultaneos)")
print("=" * 60)

invalid_payload = {
    "prompt": "hola",           # Muy corto (min_length=5)
    "max_tokens": "muchos",      # Tipo incorrecto (esperamos int)
    "temperature": 4.2,          # Fuera de rango (max 2.0)
}

try:
    InferenceRequest(**invalid_payload)
except ValidationError as exc:
    print(f"Se encontraron {exc.error_count()} errores:\n")
    for i, err in enumerate(exc.errors(), 1):
        print(f"  Error {i}:")
        print(f"    Campo: {err['loc']}")
        print(f"    Mensaje: {err['msg']}")
        print(f"    Tipo: {err['type']}")
        print()

---

## Seccion 2: Validacion de Features con Custom Validators

Las restricciones basicas de `Field()` cubren muchos casos, pero a veces necesitamos
**reglas de negocio** mas complejas. Para eso usamos `@field_validator`.

### Cuando usar cada mecanismo

```
  Restriccion simple          Regla de negocio          Validacion entre campos
  (rango, tipo, longitud)     (logica custom)           (campo A depende de B)
         |                          |                           |
         v                          v                           v
    Field(ge=0, le=120)     @field_validator("campo")    @model_validator(mode="after")
```

`@field_validator` recibe el valor **de un campo individual** y puede:
- Transformarlo (limpiar, normalizar).
- Rechazarlo (lanzar `ValueError`).
- Retornarlo validado.

In [ ]:
# --- Modelo: FeatureRow con custom validator ---

class FeatureRow(BaseModel):
    """Una fila de features para un modelo de deteccion de fraude."""
    age: int = Field(ge=0, le=120, description="Edad del usuario")
    monthly_income_usd: float = Field(ge=0, description="Ingreso mensual en USD")
    tenure_months: int = Field(ge=0, description="Meses como cliente")

    @field_validator("monthly_income_usd")
    @classmethod
    def reject_unrealistic_income(cls, value: float) -> float:
        """Rechaza ingresos mensuales irrealisticamente altos."""
        if value > 1_000_000:
            raise ValueError(
                f"monthly_income_usd={value:,.0f} parece fuera de rango realista "
                f"(maximo aceptado: 1,000,000)"
            )
        return value


# --- Demo: features VALIDAS ---
print("=" * 60)
print("FEATURES VALIDAS")
print("=" * 60)

valid_features = FeatureRow(age=29, monthly_income_usd=820.5, tenure_months=1)
print(f"Features: {valid_features.model_dump()}")
print()

# --- Demo: features INVALIDAS (custom validator) ---
print("=" * 60)
print("FEATURES INVALIDAS (ingreso irreal)")
print("=" * 60)

try:
    FeatureRow(age=31, monthly_income_usd=2_500_000, tenure_months=4)
except ValidationError as exc:
    for err in exc.errors():
        print(f"  Campo: {err['loc']}")
        print(f"  Mensaje: {err['msg']}")
        print(f"  Input rechazado: {err.get('input', 'N/A')}")
        print()

---

## Seccion 3: Validacion de Output del Modelo

No solo necesitamos validar lo que **entra** al pipeline, tambien lo que **sale**.

Un modelo de ML puede producir:
- Un **label** que deberia ser uno de un conjunto fijo de valores (`Literal`).
- Un **score** que deberia estar entre 0 y 1.
- Una **explicacion** que deberia tener cierta longitud minima.

Si alguno de estos contratos se rompe, queremos saberlo **antes** de enviar la respuesta al cliente.

In [ ]:
# --- Modelo: FraudPrediction ---

class FraudPrediction(BaseModel):
    """Resultado validado de un modelo de deteccion de fraude."""
    label: Literal["fraud", "not_fraud"]
    score: float = Field(ge=0.0, le=1.0, description="Probabilidad de fraude")
    rationale: str = Field(min_length=10, description="Explicacion de la prediccion")


def fake_model_predict(features: FeatureRow) -> FraudPrediction:
    """Modelo simplificado solo para demo educativa."""
    if features.monthly_income_usd < 1000 and features.tenure_months < 3:
        return FraudPrediction(
            label="fraud",
            score=0.81,
            rationale="Patron de riesgo: bajo ingreso y poca antiguedad.",
        )
    return FraudPrediction(
        label="not_fraud",
        score=0.22,
        rationale="No se observan senales fuertes de fraude en las features.",
    )


# --- Pipeline: validar features -> predecir -> validar output ---
print("=" * 60)
print("PIPELINE COMPLETO: Features -> Modelo -> Prediccion")
print("=" * 60)

# Paso 1: Validar features
features = FeatureRow(age=29, monthly_income_usd=820.5, tenure_months=1)
print(f"\n[Paso 1] Features validadas:")
print(f"  {features.model_dump()}")

# Paso 2: Ejecutar modelo
prediction = fake_model_predict(features)
print(f"\n[Paso 2] Prediccion generada y validada:")
print(f"  Label:     {prediction.label}")
print(f"  Score:     {prediction.score}")
print(f"  Rationale: {prediction.rationale}")

# Paso 3: Serializar para respuesta
print(f"\n[Paso 3] JSON listo para enviar al cliente:")
print(f"  {prediction.model_dump_json(indent=2)}")

---

## Seccion 4: Pipeline Completo con Manejo de Errores

Ahora unimos todo en una funcion que toma un `dict` crudo y ejecuta el pipeline completo
con manejo robusto de errores.

```
    dict crudo                                               dict / error
        |                                                        ^
        v                                                        |
  +------------------------------------------------------------------+
  |                    run_fraud_pipeline()                           |
  |                                                                  |
  |   raw_dict  -->  FeatureRow(**raw_dict)  -->  fake_model_predict  |
  |                       |                            |             |
  |                  ValidationError?             FraudPrediction    |
  |                       |                            |             |
  |                  return error              return .model_dump()  |
  +------------------------------------------------------------------+
```

**Objetivo:** una sola funcion que nunca lanza excepciones no manejadas.

In [ ]:
# --- Pipeline completo con manejo de errores ---

def run_fraud_pipeline(raw_data: dict) -> dict:
    """
    Ejecuta el pipeline de deteccion de fraude.
    
    Retorna:
        dict con "status": "ok" y la prediccion, o
        dict con "status": "error" y los detalles del error.
    """
    # Paso 1: Validar features de entrada
    try:
        features = FeatureRow(**raw_data)
    except ValidationError as exc:
        return {
            "status": "error",
            "stage": "input_validation",
            "error_count": exc.error_count(),
            "errors": [
                {"campo": str(e["loc"]), "mensaje": e["msg"]}
                for e in exc.errors()
            ],
        }

    # Paso 2: Ejecutar modelo y validar output
    try:
        prediction = fake_model_predict(features)
    except ValidationError as exc:
        return {
            "status": "error",
            "stage": "output_validation",
            "error_count": exc.error_count(),
            "errors": [
                {"campo": str(e["loc"]), "mensaje": e["msg"]}
                for e in exc.errors()
            ],
        }

    # Paso 3: Exito
    return {
        "status": "ok",
        "input": features.model_dump(),
        "prediction": prediction.model_dump(),
    }


# --- Probar con 3 casos ---
test_cases = [
    {
        "nombre": "Caso 1: Valido (cliente joven, bajo ingreso)",
        "data": {"age": 22, "monthly_income_usd": 500.0, "tenure_months": 2},
    },
    {
        "nombre": "Caso 2: Edge case (valores en el limite)",
        "data": {"age": 120, "monthly_income_usd": 1_000_000, "tenure_months": 0},
    },
    {
        "nombre": "Caso 3: Invalido (multiples errores)",
        "data": {"age": -5, "monthly_income_usd": 2_500_000, "tenure_months": "tres"},
    },
]

for case in test_cases:
    print("=" * 60)
    print(case["nombre"])
    print("-" * 60)
    result = run_fraud_pipeline(case["data"])
    print(json.dumps(result, indent=2, ensure_ascii=False))
    print()

---

## Seccion 5: Dict vs Pydantic - Detectando Bugs Silenciosos

Esta es la seccion mas importante del notebook. Vamos a ver **exactamente** por que
usar dicts crudos en pipelines de AI/ML es peligroso.

El problema fundamental:

```
  Dict crudo                              Pydantic
  --------                                --------
  - Acepta CUALQUIER dato                 - Solo acepta datos validos
  - Errores aparecen DESPUES              - Errores aparecen EN EL MOMENTO
    (durante el entrenamiento,              (antes de que el dato entre
     la inferencia, o en produccion)         al pipeline)
  - Mensajes de error crípticos           - Mensajes claros con ubicacion
    (KeyError, TypeError, NaN)              y tipo de error
```

In [ ]:
# --- Comparacion: Dict vs Pydantic ---

# ============================================
# ENFOQUE 1: Solo dicts (SIN validacion)
# ============================================

def process_with_dict(raw_data: dict) -> dict:
    """
    Procesa features directamente desde un dict.
    NO valida nada. Los errores se propagan silenciosamente.
    """
    try:
        # Simulamos calculo de un 'risk_score' basado en features
        risk = (
            (100 - raw_data["age"]) * 0.3
            + (5000 - raw_data["monthly_income_usd"]) * 0.0005
            + (12 - raw_data["tenure_months"]) * 0.1
        )
        return {"status": "ok", "risk_score": round(risk, 4)}
    except Exception as exc:
        return {"status": "error_tardio", "tipo": type(exc).__name__, "msg": str(exc)}


# ============================================
# ENFOQUE 2: Con Pydantic (CON validacion)
# ============================================

def process_with_pydantic(raw_data: dict) -> dict:
    """
    Valida con Pydantic ANTES de procesar.
    Los errores se detectan temprano.
    """
    try:
        features = FeatureRow(**raw_data)
    except ValidationError as exc:
        return {
            "status": "error_temprano",
            "errores": [f"{e['loc']}: {e['msg']}" for e in exc.errors()],
        }

    risk = (
        (100 - features.age) * 0.3
        + (5000 - features.monthly_income_usd) * 0.0005
        + (12 - features.tenure_months) * 0.1
    )
    return {"status": "ok", "risk_score": round(risk, 4)}


# --- Datos de prueba: mezcla de buenos y malos ---
test_payloads = [
    {"desc": "Valido normal",          "data": {"age": 35, "monthly_income_usd": 3000, "tenure_months": 24}},
    {"desc": "Edad negativa",          "data": {"age": -5, "monthly_income_usd": 3000, "tenure_months": 12}},
    {"desc": "Ingreso como string",    "data": {"age": 40, "monthly_income_usd": "mucho", "tenure_months": 6}},
    {"desc": "Campo faltante",         "data": {"age": 30, "tenure_months": 10}},
]

print(f"{'Caso':<25} {'Dict (sin validar)':<45} {'Pydantic (validado)':<45}")
print("=" * 115)

for payload in test_payloads:
    dict_result = process_with_dict(payload["data"])
    pydantic_result = process_with_pydantic(payload["data"])

    # Resumir resultados para la tabla
    if dict_result["status"] == "ok":
        dict_summary = f"ok (risk={dict_result['risk_score']})  <-- puede ser BASURA"
    else:
        dict_summary = f"{dict_result['tipo']}: {dict_result['msg'][:35]}"

    if pydantic_result["status"] == "ok":
        pydantic_summary = f"ok (risk={pydantic_result['risk_score']})"
    else:
        pydantic_summary = f"RECHAZADO: {'; '.join(pydantic_result['errores'])[:35]}"

    print(f"{payload['desc']:<25} {dict_summary:<45} {pydantic_summary:<45}")

print()
print("Observa: con dicts, la 'edad negativa' produce un risk_score INCORRECTO pero NO da error.")
print("Pydantic lo detecta INMEDIATAMENTE.")

In [ ]:
# --- Stress test: 10 payloads, dict vs Pydantic ---

stress_payloads = [
    {"age": 25, "monthly_income_usd": 3500, "tenure_months": 14},     # Valido
    {"age": 0,  "monthly_income_usd": 0,    "tenure_months": 0},      # Valido (edge)
    {"age": -3, "monthly_income_usd": 2000,  "tenure_months": 5},     # Invalido: age < 0
    {"age": 45, "monthly_income_usd": -100,  "tenure_months": 20},    # Invalido: income < 0
    {"age": 130, "monthly_income_usd": 5000, "tenure_months": 36},    # Invalido: age > 120
    {"age": 55, "monthly_income_usd": 2_500_000, "tenure_months": 8}, # Invalido: income irreal
    {"age": "joven", "monthly_income_usd": 1000, "tenure_months": 3}, # Invalido: tipo incorrecto
    {"age": 38, "monthly_income_usd": 4200, "tenure_months": 48},     # Valido
    {"monthly_income_usd": 3000, "tenure_months": 12},                # Invalido: falta age
    {"age": 60, "monthly_income_usd": 8000, "tenure_months": -2},     # Invalido: tenure < 0
]

dict_ok = 0
dict_error_tardio = 0
dict_bug_silencioso = 0
pydantic_ok = 0
pydantic_rechazado = 0

for i, payload in enumerate(stress_payloads):
    # Dict approach
    dict_res = process_with_dict(payload)
    if dict_res["status"] == "ok":
        # Verificamos si el dato REALMENTE era valido
        pydantic_check = process_with_pydantic(payload)
        if pydantic_check["status"] == "ok":
            dict_ok += 1
        else:
            dict_bug_silencioso += 1  # Dict dijo 'ok' pero dato era invalido!
    else:
        dict_error_tardio += 1

    # Pydantic approach
    pydantic_res = process_with_pydantic(payload)
    if pydantic_res["status"] == "ok":
        pydantic_ok += 1
    else:
        pydantic_rechazado += 1

print("=" * 60)
print("STRESS TEST: 10 payloads (mezcla de validos e invalidos)")
print("=" * 60)
print()
print(f"{'Metrica':<40} {'Dict':>8} {'Pydantic':>10}")
print("-" * 60)
print(f"{'Procesados correctamente':<40} {dict_ok:>8} {pydantic_ok:>10}")
print(f"{'Errores detectados temprano':<40} {'N/A':>8} {pydantic_rechazado:>10}")
print(f"{'Errores detectados tarde (excepcion)':<40} {dict_error_tardio:>8} {'0':>10}")
print(f"{'BUGS SILENCIOSOS (dato malo, sin error)':<40} {dict_bug_silencioso:>8} {'0':>10}")
print()
print(f"Los {dict_bug_silencioso} bugs silenciosos del dict son datos invalidos que")
print(f"se procesaron sin error y produjeron resultados INCORRECTOS.")
print(f"Pydantic los hubiera rechazado inmediatamente.")

---

## Seccion 6: Patrones Avanzados

Tres patrones que aparecen frecuentemente en pipelines de produccion:

1. **Modelos anidados:** Un `BatchRequest` que contiene una lista de `InferenceRequest`.
2. **`model_validator`:** Validacion que involucra multiples campos (ej: si `temperature > 1.5`, `max_tokens` no puede ser mayor a 500).
3. **Export de JSON Schema:** Generar el esquema JSON del modelo para documentacion o integracion con OpenAPI.

In [ ]:
# --- Patron 1: Modelos anidados ---

class BatchInferenceRequest(BaseModel):
    """Batch de multiples requests de inferencia."""
    requests: list[InferenceRequest] = Field(
        min_length=1, max_length=50, description="Lista de requests individuales"
    )
    priority: Literal["low", "medium", "high"] = Field(default="medium")


print("=" * 60)
print("PATRON 1: Modelos anidados (BatchInferenceRequest)")
print("=" * 60)

batch = BatchInferenceRequest(
    requests=[
        InferenceRequest(prompt="Analiza este texto sobre redes neuronales."),
        InferenceRequest(prompt="Resume los resultados del experimento.", max_tokens=100),
    ],
    priority="high",
)
print(f"Batch con {len(batch.requests)} requests, prioridad: {batch.priority}")
for i, req in enumerate(batch.requests):
    print(f"  Request {i+1}: prompt='{req.prompt[:40]}...' max_tokens={req.max_tokens}")

print()

# --- Patron 2: model_validator (validacion cruzada entre campos) ---

class SafeInferenceRequest(BaseModel):
    """Request con validacion cruzada: alta temperatura + muchos tokens = peligro."""
    prompt: str = Field(min_length=5, max_length=2000)
    max_tokens: int = Field(default=256, ge=1, le=4096)
    temperature: float = Field(default=0.2, ge=0.0, le=2.0)

    @model_validator(mode="after")
    def check_high_temp_low_tokens(self) -> SafeInferenceRequest:
        """Si temperature > 1.5, limitar max_tokens para evitar outputs muy largos y ruidosos."""
        if self.temperature > 1.5 and self.max_tokens > 500:
            raise ValueError(
                f"Combinacion peligrosa: temperature={self.temperature} con "
                f"max_tokens={self.max_tokens}. Con temperature > 1.5, "
                f"max_tokens debe ser <= 500 para evitar outputs ruidosos."
            )
        return self


print("=" * 60)
print("PATRON 2: model_validator (validacion cruzada)")
print("=" * 60)

# Valido: alta temperatura pero pocos tokens
safe_req = SafeInferenceRequest(
    prompt="Genera ideas creativas para nombres de producto.",
    temperature=1.8,
    max_tokens=200,
)
print(f"Valido: temp={safe_req.temperature}, tokens={safe_req.max_tokens}")

# Invalido: alta temperatura Y muchos tokens
try:
    SafeInferenceRequest(
        prompt="Genera un ensayo largo sobre inteligencia artificial.",
        temperature=1.8,
        max_tokens=2000,
    )
except ValidationError as exc:
    for err in exc.errors():
        print(f"Rechazado: {err['msg']}")

print()

# --- Patron 3: Export de JSON Schema ---

print("=" * 60)
print("PATRON 3: JSON Schema (para documentacion y OpenAPI)")
print("=" * 60)

schema = InferenceRequest.model_json_schema()
print(json.dumps(schema, indent=2, ensure_ascii=False))
print()
print("Este schema se puede usar directamente en:")
print("  - Documentacion de API (OpenAPI/Swagger)")
print("  - Validacion en el frontend")
print("  - Generacion automatica de formularios")

---

## Seccion 7: Serializacion y Deserializacion

En pipelines reales, los datos llegan como **JSON** (desde APIs, archivos, colas de mensajes).
Pydantic facilita la conversion entre JSON y objetos validados.

In [ ]:
# --- Serializacion y deserializacion ---

print("=" * 60)
print("SERIALIZACION: Objeto -> JSON")
print("=" * 60)

features = FeatureRow(age=35, monthly_income_usd=4500.0, tenure_months=24)

# model_dump() -> dict
as_dict = features.model_dump()
print(f"model_dump() -> {type(as_dict).__name__}: {as_dict}")

# model_dump_json() -> str JSON
as_json = features.model_dump_json(indent=2)
print(f"\nmodel_dump_json():\n{as_json}")

print()
print("=" * 60)
print("DESERIALIZACION: JSON -> Objeto validado")
print("=" * 60)

# model_validate_json() -> FeatureRow (desde JSON string)
json_string = '{"age": 42, "monthly_income_usd": 6200.0, "tenure_months": 36}'
from_json = FeatureRow.model_validate_json(json_string)
print(f"Desde JSON string: {from_json}")

# model_validate() -> FeatureRow (desde dict)
from_dict = FeatureRow.model_validate({"age": 50, "monthly_income_usd": 7000, "tenure_months": 48})
print(f"Desde dict:        {from_dict}")

print()

# JSON invalido tambien se detecta
print("JSON invalido:")
try:
    FeatureRow.model_validate_json('{"age": -10, "monthly_income_usd": "no_se", "tenure_months": 5}')
except ValidationError as exc:
    for err in exc.errors():
        print(f"  {err['loc']}: {err['msg']}")

---

## Seccion 8: Resumen de la API de Pydantic

Referencia rapida de los metodos y decoradores que usamos:

| Metodo / Decorador | Que hace | Ejemplo |
|---|---|---|
| `Field(ge=0, le=100)` | Restriccion numerica en campo | `age: int = Field(ge=0, le=120)` |
| `Field(min_length=5)` | Restriccion de longitud de string | `prompt: str = Field(min_length=5)` |
| `Field(default=X)` | Valor por defecto | `temperature: float = Field(default=0.2)` |
| `@field_validator` | Validacion custom de un campo | Rechazar ingresos > 1M |
| `@model_validator` | Validacion cruzada entre campos | temp alta + muchos tokens |
| `Literal["a", "b"]` | Solo acepta valores del conjunto | `label: Literal["fraud", "not_fraud"]` |
| `.model_dump()` | Convierte a dict | `features.model_dump()` |
| `.model_dump_json()` | Convierte a JSON string | `features.model_dump_json()` |
| `.model_validate()` | Crea desde dict con validacion | `FeatureRow.model_validate(d)` |
| `.model_validate_json()` | Crea desde JSON string con validacion | `FeatureRow.model_validate_json(s)` |
| `.model_json_schema()` | Exporta JSON Schema | Para OpenAPI/Swagger |

---

## Ejercicios

Completa los siguientes ejercicios para afianzar lo aprendido. Cada uno incluye instrucciones
detalladas y un esqueleto de codigo para que completes.

In [ ]:
# ==========================================================
# EJERCICIO 1: Modelos para un Pipeline de Entrenamiento
# ==========================================================
#
# Crea dos modelos Pydantic:
#
# 1. TrainingConfig:
#    - learning_rate: float entre 1e-6 y 1.0
#    - epochs: int entre 1 y 1000
#    - batch_size: int que DEBE ser potencia de 2 (usa @field_validator)
#      Hint: un numero es potencia de 2 si (n & (n - 1)) == 0 y n > 0
#    - optimizer: Literal["adam", "sgd", "adamw"]
#
# 2. TrainingResult:
#    - final_loss: float >= 0
#    - accuracy: float entre 0.0 y 1.0
#    - duration_seconds: float > 0
#    - config: TrainingConfig  (modelo anidado!)
#
# Prueba:
#   a) Crea un TrainingConfig valido y uno invalido (batch_size=100).
#   b) Crea un TrainingResult completo.
#   c) Exporta el JSON Schema de TrainingResult.
# ==========================================================

class TrainingConfig(BaseModel):
    # Tu codigo aqui
    pass


class TrainingResult(BaseModel):
    # Tu codigo aqui
    pass


# Pruebas (descomenta cuando completes los modelos):
# config = TrainingConfig(learning_rate=0.001, epochs=50, batch_size=32, optimizer="adam")
# print("Config valido:", config.model_dump())
#
# try:
#     TrainingConfig(learning_rate=0.001, epochs=50, batch_size=100, optimizer="adam")
# except ValidationError as exc:
#     print("Batch size invalido:", exc.errors()[0]["msg"])
#
# result = TrainingResult(
#     final_loss=0.023, accuracy=0.97, duration_seconds=342.5, config=config
# )
# print("\nResultado completo:")
# print(result.model_dump_json(indent=2))

In [ ]:
# ==========================================================
# EJERCICIO 2: Pipeline de Analisis de Sentimiento
# ==========================================================
#
# Crea:
#
# 1. SentimentAnalysisRequest:
#    - text: str (min_length=10, max_length=5000)
#    - language: Literal["es", "en", "pt"] con default "es"
#    - include_confidence: bool con default True
#
# 2. SentimentAnalysisResponse:
#    - sentiment: Literal["positive", "negative", "neutral"]
#    - confidence: float entre 0.0 y 1.0
#    - original_text: str
#    - @model_validator: si confidence < 0.5, sentiment debe ser "neutral"
#      (un modelo con poca confianza no deberia dar una opinion fuerte)
#
# 3. Funcion fake_sentiment_analysis(request) -> SentimentAnalysisResponse
#    que analice el texto de forma simplificada.
#
# 4. Procesa un batch de 5 textos (algunos validos, algunos invalidos).
#    Imprime resultados y errores.
# ==========================================================

class SentimentAnalysisRequest(BaseModel):
    # Tu codigo aqui
    pass


class SentimentAnalysisResponse(BaseModel):
    # Tu codigo aqui
    pass


def fake_sentiment_analysis(request: SentimentAnalysisRequest) -> SentimentAnalysisResponse:
    # Tu codigo aqui
    pass


# batch_texts = [
#     {"text": "Este producto es increible, me encanta!", "language": "es"},
#     {"text": "Pesimo servicio, nunca mas vuelvo.", "language": "es"},
#     {"text": "corto"},  # invalido: muy corto
#     {"text": "El producto esta bien, nada especial.", "language": "es"},
#     {"text": "Great product, highly recommended!", "language": "en"},
# ]

In [ ]:
# ==========================================================
# EJERCICIO 3: Pipeline Validado para un Dominio de tu Eleccion
# ==========================================================
#
# Construye un pipeline completo para UNO de estos dominios
# (o inventa el tuyo propio):
#
# Opcion A: Clasificacion de imagenes
#   - ImageClassificationRequest: image_url, model_name, top_k
#   - ImageClassificationResponse: predictions (lista de {label, score})
#
# Opcion B: Recomendacion de productos
#   - UserProfile: user_id, age, interests (lista), purchase_history_count
#   - RecommendationResponse: products (lista), confidence, strategy
#
# Opcion C: Tu propio dominio
#   - Define al menos 2 modelos (input y output)
#   - Incluye al menos 1 field_validator y 1 model_validator
#
# Requisitos minimos:
#   1. Modelo de input con al menos 4 campos y validaciones
#   2. Modelo de output con Literal types y restricciones
#   3. Funcion de pipeline que valida input -> procesa -> valida output
#   4. Manejo de errores con mensajes claros
#   5. Prueba con al menos 3 casos (valido, edge case, invalido)
#   6. Exporta el JSON Schema de tu modelo principal
# ==========================================================

# Tu pipeline aqui
pass

---

## Checklist de Consolidacion

Antes de cerrar este notebook, verifica que puedes responder "si" a cada punto:

- [ ] **Se crear un modelo Pydantic** con `Field()` para definir restricciones de tipo, rango y longitud.
- [ ] **Se usar `@field_validator`** para agregar reglas de validacion personalizadas a un campo.
- [ ] **Se usar `@model_validator`** para validaciones que involucran multiples campos.
- [ ] **Entiendo la diferencia entre dict y Pydantic** y puedo explicar por que los dicts producen bugs silenciosos.
- [ ] **Se atrapar `ValidationError`** e iterar sobre `.errors()` para generar mensajes de error utiles.
- [ ] **Se serializar/deserializar** modelos con `model_dump()`, `model_dump_json()`, `model_validate()` y `model_validate_json()`.
- [ ] **Se disenar contratos de datos** para un pipeline input -> procesamiento -> output.
- [ ] **Se exportar JSON Schema** con `model_json_schema()` para documentacion o integracion.

---

### Conclusion

> **Pydantic convierte errores silenciosos en errores explicitos y accionables.**

En pipelines de AI/ML, donde los datos fluyen a traves de multiples etapas y los errores
pueden propagarse silenciosamente durante horas o dias, la validacion estructurada no es
un lujo: es una necesidad.

Lo que aprendimos hoy:
- Pydantic declara **que forma deben tener los datos** (contrato).
- Los errores se reportan **en el momento exacto** en que ocurren, no minutos u horas despues.
- La combinacion de `Field()`, `@field_validator` y `@model_validator` cubre desde restricciones simples hasta reglas de negocio complejas.
- El JSON Schema exportado permite integrar la validacion con el resto del ecosistema (APIs, frontend, documentacion).

---

### Proximos pasos

**Notebook 02: Patrones de Produccion** - Veremos como aplicar estos conceptos en:
- APIs con FastAPI + Pydantic
- Configuracion de experimentos de ML
- Logging estructurado con modelos validados
- Testing automatizado de contratos de datos